### This notebook Contains pytorch implementation of both the Word2Vec models, CBOW and Skipgram:

[Word2Vec arXiv:1301.3781](https://arxiv.org/pdf/1301.3781)

In [ ]:
import json
d = {"username":"<USER_NAME>","key":"<API_KEY>"}

with open('kaggle.json','w') as f:
  json.dump(d,f)

!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 777 ~/.kaggle/kaggle.json
!kaggle datasets download -d gunnvant/game-of-thrones-srt
!unzip game-of-thrones-srt.zip

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import pandas as pd
!pip install pandarallel
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
stop_words = set(stopwords.words("english"))
import re

def clean_text(text):
    try:
      text=str(text).lower()
    except:
      pass
    cleaned_text = re.sub(r'<[^>]*>', '', text)
    cleaned_text = re.sub(r'\S+@\S+', '', cleaned_text)
    cleaned_text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', cleaned_text)
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
    cleaned_text = re.sub(r'\n', ' ', cleaned_text)
    cleaned_text = re.sub(r'\t', ' ', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    # print(cleaned_text)
    return cleaned_text

def stop_word_remove(text):
  a,res = text.split(),list()
  for i in a:
    if i not in stop_words:
      res.append(i)
  return ' '.join(res)

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lem(l):
  l = l.split(' ')
  res = list()
  for i in l:
     res.append(lemmatizer.lemmatize(i))
  return " ".join(res)

In [ ]:
import pandas as pd

df = pd.read_json("season1.json")
# print(df.head())

for i in df.columns:
  df[i] = df[i].apply(clean_text)
  df[i] = df[i].apply(lem)

In [ ]:
df.head()

,Game Of Thrones S01E01 Winter Is Coming.srt,Game Of Thrones S01E02 The Kingsroad.srt,Game Of Thrones S01E03 Lord Snow.srt,"Game Of Thrones S01E04 Cripples, Bastards, And Broken Things.srt",Game Of Thrones S01E05 The Wolf And The Lion.srt,Game Of Thrones S01E06 A Golden Crown.srt,Game Of Thrones S01E07 You Win Or You Die.srt,Game Of Thrones S01E08 The Pointy End.srt,Game Of Thrones S01E09 Baelor.srt,Game Of Thrones S01E10 Fire And Blood.srt
1,easy boy,you need to drink child,welcome lord stark,the little lord been dreaming again,doe ser hugh have any family in the capital,your pardon your grace,summoned to court to answer for the crime,yah left high left low,youve seen better day my lord,look at me look at me
10,our order were to track the wildlings,the dothraki believe that one day,sturdy old thing,hodor help bran down the hall,but who hold the straw,he butchered my men,go on say something clever,go back to your room,ive given up trying to guess,it told me to come with him so i did
100,right give it here no,such a little thing,thats not true you killed the beast,were done for today go clean the armoury,in garb of red and gold,drop the blade,your sister wa a corpse and i wa a living girl,we should burn them snow not wrong my lord,oh you too,this one here
101,put away your blade,a bird without feather,you only spared the girl because of the love y...,thats all youre good for,this isnt the kingsroad,no dont do it,and he loved her more than me,fire will do for them the wildling way,you see that,look at it and see what happens to traitor
102,i take order from your father not you please f...,they came to take his body away,i didnt i,well fought piss off,you said we were riding for winterfell i did o...,are you all right,when the king return from his hunt ill tell hi...,i want maester aemon to examine them first,fifteen she is,you promised to be merciful i wa


In [ ]:
df.columns

Index(['Game Of Thrones S01E01 Winter Is Coming.srt',
       'Game Of Thrones S01E02 The Kingsroad.srt',
       'Game Of Thrones S01E03 Lord Snow.srt',
       'Game Of Thrones S01E04 Cripples, Bastards, And Broken Things.srt',
       'Game Of Thrones S01E05 The Wolf And The Lion.srt',
       'Game Of Thrones S01E06 A Golden Crown.srt',
       'Game Of Thrones S01E07 You Win Or You Die.srt',
       'Game Of Thrones S01E08 The Pointy End.srt',
       'Game Of Thrones S01E09 Baelor.srt',
       'Game Of Thrones S01E10 Fire And Blood.srt'],
      dtype='object')

In [ ]:
sentences = [' '.join(col) for col in df.values.T]
unique_words = list(set((' '.join(sentences)).split(" ")))

In [ ]:
# def OHE(text):
#   ohe_rep = list()
#   for i in text.split():
#     ohe_rep.append(unique_words.index(i)+1)
#   if len(ohe_rep)<3:
#     ohe_rep = [0]*(3-len(ohe_rep)) + ohe_rep
#   return ohe_rep

# for i in df.columns:
#   df["OHE_"+i] = df[i].apply(OHE)

In [ ]:
df.head(5)

,Game Of Thrones S01E01 Winter Is Coming.srt,Game Of Thrones S01E02 The Kingsroad.srt,Game Of Thrones S01E03 Lord Snow.srt,"Game Of Thrones S01E04 Cripples, Bastards, And Broken Things.srt",Game Of Thrones S01E05 The Wolf And The Lion.srt,Game Of Thrones S01E06 A Golden Crown.srt,Game Of Thrones S01E07 You Win Or You Die.srt,Game Of Thrones S01E08 The Pointy End.srt,Game Of Thrones S01E09 Baelor.srt,Game Of Thrones S01E10 Fire And Blood.srt
1,easy boy,you need to drink child,welcome lord stark,the little lord been dreaming again,doe ser hugh have any family in the capital,your pardon your grace,summoned to court to answer for the crime,yah left high left low,youve seen better day my lord,look at me look at me
10,our order were to track the wildlings,the dothraki believe that one day,sturdy old thing,hodor help bran down the hall,but who hold the straw,he butchered my men,go on say something clever,go back to your room,ive given up trying to guess,it told me to come with him so i did
100,right give it here no,such a little thing,thats not true you killed the beast,were done for today go clean the armoury,in garb of red and gold,drop the blade,your sister wa a corpse and i wa a living girl,we should burn them snow not wrong my lord,oh you too,this one here
101,put away your blade,a bird without feather,you only spared the girl because of the love y...,thats all youre good for,this isnt the kingsroad,no dont do it,and he loved her more than me,fire will do for them the wildling way,you see that,look at it and see what happens to traitor
102,i take order from your father not you please f...,they came to take his body away,i didnt i,well fought piss off,you said we were riding for winterfell i did o...,are you all right,when the king return from his hunt ill tell hi...,i want maester aemon to examine them first,fifteen she is,you promised to be merciful i wa


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [ ]:
import torch
import torch.nn as nn

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMDEDDING_DIM = 100

raw_text = (" ".join(sentences)).split()

vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word:ix for ix, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

In [ ]:
class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()

        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        # self.activation_function2 = nn.LogSoftmax(dim = -1)
        self.activation_function2 = nn.Sigmoid()


    def forward(self, inputs):
        embeds = self.embeddings(inputs).sum(dim=1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]]).to('cuda')
        return self.embeddings(word).view(1,-1)


model = CBOW(vocab_size, EMDEDDING_DIM).to('cuda')

In [ ]:
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

In [ ]:
# full batched dataset
context_batch= list()
ground_batch= list()

for context, target in data:
  context_vector = make_context_vector(context, word_to_ix)
  ground_vector = torch.zeros(vocab_size)
  ground_vector[word_to_ix[target]] =1
  context_batch.append(context_vector)
  ground_batch.append(ground_vector)

context_tensor = torch.stack(context_batch).to('cuda')
ground_tensor = torch.stack(ground_batch).to('cuda')

In [ ]:
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)

#TRAINING
for epoch in range(500):
    total_loss = 0

    log_probs = model(context_tensor).to('cuda')
    total_loss += loss_function(log_probs, ground_tensor)

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch} , loss: {total_loss}")

In [ ]:
import random
#TESTING
context = random.sample(list(word_to_ix.keys()), 4)
context_batch = list()
context_vector = make_context_vector(context, word_to_ix)
context_batch.append(context_vector)
context_tensor = torch.stack(context_batch).to('cuda')
a = model(context_tensor)

print(f'Raw text: {" ".join(raw_text)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')
word = input("Enter a word to generate embedding for:") # must be in vocab, father
print(model.get_word_emdedding(word))

# Skip gram


In [ ]:
def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    - words: list of words
    - return: 2 dictionaries, vocab_to_int, int_to_vocab
    """
    word_counts = Counter(words)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True) # descending freq order
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

In [ ]:
from collections import Counter
import numpy as np
import random
import torch
from torch import nn
from torch.functional import F
from torch import optim

words = (" ".join(sentences)).split()
vocab_to_int, int_to_vocab = create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

In [ ]:
# get a random range of 1-5 context word targets on the left & right of the word input with index 'idx' from 'words'
def get_targets(words, idx, window_size=5):
    R = random.randint(1, 5)
    start = max(0,idx-R)
    end = min(idx+R,len(words)-1)
    targets = words[start:idx] + words[idx+1:end+1] # +1 since doesn't include this idx

    return targets

def get_batches(words, batch_size, window_size = 5):

    for i in range(0, len(words), batch_size):
        curr = words[i:i + batch_size]   # current batch
        batch_x, batch_y = [], []

        for ii in range(len(curr)):
            x = [curr[ii]]
            y = get_targets(curr, ii)
            batch_x.extend(x * len(y))
            batch_y.extend(y)

        yield batch_x, batch_y

In [ ]:
class skipgram(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()

        self.n_vocab = n_vocab
        self.n_embed = n_embed

        self.in_embed = nn.Embedding(n_vocab, n_embed)
        self.out_embed = nn.Embedding(n_vocab, n_embed)

        # Initialize both embedding tables with uniform distribution
        self.in_embed.weight.data.uniform_(-1, 1)
        self.out_embed.weight.data.uniform_(-1, 1)


    def forward_input(self, input_words):
        input_vectors = self.in_embed(input_words)
        return input_vectors  # input vector embeddings


    def forward_target(self, output_words):
        output_vectors = self.out_embed(output_words)
        return output_vectors  # output vector embeddings


class skipgram_loss(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, input_vectors,output_vectors):

    batch_size, embed_size = input_vectors.shape

    input_vectors = input_vectors.view(batch_size, embed_size, 1)   # batch of column vectors
    output_vectors = output_vectors.view(batch_size, 1, embed_size) # batch of row vectors

    out_loss = torch.bmm(output_vectors, input_vectors).sigmoid().log().squeeze()
    return -(out_loss).mean()  # average batch loss

In [ ]:
embedding_dim = 100
model = skipgram(len(vocab_to_int), embedding_dim)
criterion = skipgram_loss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

n_epochs = 100
batch_size=512
device = 'cuda'

In [ ]:
model.to(device)
for epoch in range(n_epochs):
    for inputs, targets in get_batches(int_words, batch_size=batch_size):
        inputs = torch.LongTensor(inputs).to(device)
        targets = torch.LongTensor(targets).to(device)

        embedded_input_words = model.forward_input(inputs)
        embedded_target_words = model.forward_target(targets)

        loss = criterion(embedded_input_words, embedded_target_words)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: {}/{}".format((epoch+1), n_epochs),"Loss: {:.4f}".format(loss.item()))

In [ ]:
input_word = torch.tensor([vocab_to_int[input()]]).to('cuda')
input_embedding = model.forward_input(input_word)
print(input_embedding)

father
tensor([[-0.2261,  0.7660, -0.5967,  0.8860, -0.2949,  0.4329,  0.6273,  0.6617,
          0.5013, -0.0425,  0.5608,  0.8947, -0.7771,  0.5534, -0.8137, -0.0393,
         -0.4253, -0.7596, -0.5629,  0.2310, -1.4545,  0.1843,  0.0640,  0.3694,
          0.0926, -0.3834, -0.7510,  0.0471, -0.2970,  0.8468,  0.0801, -0.6274,
         -1.2727,  0.7704,  0.0247, -0.8894,  1.0190,  0.0716,  1.2534,  0.0765,
          1.0813,  0.4007,  0.1370, -0.5045, -1.1305, -0.1924,  0.5259, -0.5743,
          0.2333,  0.2299,  0.6738, -0.7280, -0.5187, -0.6225,  0.4251,  0.2973,
         -0.2311,  0.6139,  0.6722,  0.0980, -0.0300,  0.0716,  0.1413, -0.4129,
         -0.3231, -0.7833,  0.0478,  0.0996, -0.9909, -0.8515,  0.5675,  0.5749,
         -0.4450,  0.8691, -0.5062, -0.7149, -0.2534,  0.4175, -0.1417,  0.6910,
          0.4539, -0.1746, -0.5364, -0.5553, -0.0111,  1.2728, -0.5924, -0.1228,
         -0.5155,  0.1480,  0.8355, -0.5499, -0.3024,  0.7636,  0.4661, -0.5091,
         -0.4167, -0.